In [2]:
'''
Patrick Ballou
ID: 801130521
ECGR 4105
Homework 6
Problem 2
'''

'\nPatrick Ballou\nID: 801130521\nECGR 4105\nHomework 6\nProblem 2\n'

In [3]:
import torch
from torch import cuda
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
import datetime

In [4]:
data_path = 'C:/Users/patri/Documents/School_Fall_2022/ECGR_4105/Homework/homework_6'
#normalized
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = 'cuda'
cuda.get_device_name()

'Quadro T2000'

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [97]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs, labels = imgs.to(device=device), labels.to(device=device)
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    print("{} Epoch {}, Training loss {}".format(datetime.datetime.now(), epoch,loss_train / len(train_loader)))


In [98]:
l_r = .008
num_epochs = 300

In [99]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=128, shuffle=True)
model_a = Net().to(device=device)
optimizer = torch.optim.SGD(model_a.parameters(), lr=l_r)
loss_fn = nn.CrossEntropyLoss()

training_loop(num_epochs, optimizer, model_a, loss_fn, train_loader)

2022-12-10 22:08:06.680163 Epoch 1, Training loss 2.1979545046911215
2022-12-10 22:08:20.999803 Epoch 2, Training loss 1.9901536742744543
2022-12-10 22:08:35.046300 Epoch 3, Training loss 1.8898709930117479
2022-12-10 22:08:49.951607 Epoch 4, Training loss 1.801148413392284
2022-12-10 22:09:04.162427 Epoch 5, Training loss 1.7086237025687763
2022-12-10 22:09:17.786948 Epoch 6, Training loss 1.6298809902137503
2022-12-10 22:09:32.236321 Epoch 7, Training loss 1.5677374232455592
2022-12-10 22:09:47.156629 Epoch 8, Training loss 1.5179161183974321
2022-12-10 22:10:01.271216 Epoch 9, Training loss 1.4756473177839118
2022-12-10 22:10:15.797970 Epoch 10, Training loss 1.4370024292670247
2022-12-10 22:10:30.462755 Epoch 11, Training loss 1.4003242754265475
2022-12-10 22:10:44.658084 Epoch 12, Training loss 1.3669757870457055
2022-12-10 22:10:58.969155 Epoch 13, Training loss 1.3368382127693548
2022-12-10 22:11:12.816422 Epoch 14, Training loss 1.3108555907788484
2022-12-10 22:11:26.728299 Epo

In [105]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=128, shuffle=False)
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model_a(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print("Accuracy: {:.2f}".format(correct / total))

Accuracy: 0.66


In [106]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(8, 3, kernel_size=3, padding=1)
        self.act3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [107]:
l_r = .02
num_epochs = 300

In [108]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=256, shuffle=True)
model_b = Net().to(device=device)
optimizer = torch.optim.SGD(model_b.parameters(), lr=l_r)
loss_fn = nn.CrossEntropyLoss()

training_loop(num_epochs, optimizer, model_b, loss_fn, train_loader)

2022-12-10 23:28:30.487254 Epoch 1, Training loss 2.205159378295042
2022-12-10 23:28:43.116394 Epoch 2, Training loss 1.94727153255015
2022-12-10 23:28:55.956614 Epoch 3, Training loss 1.7649597884440908
2022-12-10 23:29:07.961058 Epoch 4, Training loss 1.6282386335791375
2022-12-10 23:29:20.687763 Epoch 5, Training loss 1.5353622661561381
2022-12-10 23:29:33.986395 Epoch 6, Training loss 1.4670775411080341
2022-12-10 23:29:46.892074 Epoch 7, Training loss 1.418908684837575
2022-12-10 23:29:59.886462 Epoch 8, Training loss 1.372822064526227
2022-12-10 23:30:12.591734 Epoch 9, Training loss 1.341478488274983
2022-12-10 23:30:25.580109 Epoch 10, Training loss 1.3129363175557585
2022-12-10 23:30:38.837057 Epoch 11, Training loss 1.2900027955064968
2022-12-10 23:30:51.707133 Epoch 12, Training loss 1.2732123100027746
2022-12-10 23:31:04.532012 Epoch 13, Training loss 1.25021501706571
2022-12-10 23:31:17.716773 Epoch 14, Training loss 1.229579393352781
2022-12-10 23:31:30.262882 Epoch 15, T

In [114]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=256, shuffle=False)
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = model_b(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print("Accuracy: {:.2f}".format(correct / total))

Accuracy: 0.60
